In [ ]:
'''a FastAPI /action system that matches the Streamlit “Glass-Box Mission Control” UI, 
supports Engineer View (run L0–L9), HITL approvals, and background execution with real-time state polling.'''

from __future__ import annotations

import os
from pathlib import Path
from datetime import datetime

# Auto-chdir to repo root even from notebooks_v2
ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks_v2":
    os.chdir(ROOT.parent)
ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Not at repo root. CWD={ROOT}"
print("✅ CWD =", ROOT)

def backup_write(rel_path: str, content: str) -> None:
    p = ROOT / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

# ---------- src/careeragent/api/request_models.py ----------
backup_write("src/careeragent/api/request_models.py", r'''
from __future__ import annotations

from typing import Any, Dict, Optional
from pydantic import BaseModel, Field, ConfigDict


class AnalyzeResponse(BaseModel):
    """
    Description: Response for /analyze.
    Layer: L8
    Input: run_id + status
    Output: JSON response
    """
    model_config = ConfigDict(extra="ignore")
    run_id: str
    status: str


class ActionRequest(BaseModel):
    """
    Description: Generic action request for /action/{run_id}.
    Layer: L5
    Input: action_type + payload
    Output: used by API handlers
    """
    model_config = ConfigDict(extra="ignore")
    action_type: str = Field(..., description="execute_layer|approve_ranking|reject_ranking|approve_drafts|reject_drafts|approve_job|reject_job|resume_cleanup_submit|...")
    payload: Dict[str, Any] = Field(default_factory=dict)


class StatusResponse(BaseModel):
    """
    Description: Response for /status/{run_id}.
    Layer: L8
    Input: run_id
    Output: full state snapshot
    """
    model_config = ConfigDict(extra="allow")
''')

# ---------- src/careeragent/api/run_manager_service.py ----------
backup_write("src/careeragent/api/run_manager_service.py", r'''
from __future__ import annotations

import asyncio
import json
import threading
from pathlib import Path
from typing import Any, Dict, Optional
from uuid import uuid4
from datetime import datetime, timezone

from careeragent.services.db_service import SqliteStateStore

# Optional artifacts root helper
try:
    from careeragent.config import artifacts_root  # type: ignore
except Exception:  # fallback
    def artifacts_root() -> Path:
        return Path("src/careeragent/artifacts").resolve()

# Tool-resilient nodes runner
from careeragent.langgraph.runtime_nodes import (
    run_full_pipeline,
    run_single_layer,
    approve_ranking_flow,
    approve_drafts_flow,
)


def utc_now() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")


class RunManagerService:
    """
    Description: Run manager for background LangGraph-style execution with state polling.
    Layer: L8
    Input: resume + prefs + actions
    Output: persisted state snapshots
    """

    def __init__(self) -> None:
        self._store = SqliteStateStore()
        self._lock = threading.Lock()

    def _runs_dir(self, run_id: str) -> Path:
        d = artifacts_root() / "runs" / run_id
        d.mkdir(parents=True, exist_ok=True)
        return d

    def create_run(self, *, resume_filename: str, resume_text: str, resume_bytes: bytes, preferences: Dict[str, Any]) -> Dict[str, Any]:
        """
        Description: Create new run record and persist initial state.
        Layer: L8
        Input: resume + prefs
        Output: created state snapshot
        """
        run_id = uuid4().hex
        run_dir = self._runs_dir(run_id)

        # Persist resume file for MCP/local auditing
        (run_dir / "resume_upload.bin").write_bytes(resume_bytes)
        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")

        thresholds = (preferences.get("thresholds") or {})
        if "default" in thresholds:
            # allow a global default override
            default_th = float(thresholds["default"])
            thresholds.setdefault("parser", default_th)
            thresholds.setdefault("discovery", default_th)
            thresholds.setdefault("match", default_th)
            thresholds.setdefault("draft", default_th)

        state: Dict[str, Any] = {
            "run_id": run_id,
            "status": "running",
            "pending_action": None,
            "hitl_reason": None,
            "hitl_payload": {},
            "thresholds": thresholds,
            "max_retries": int(preferences.get("max_refinements", 3)),
            "layer_retry_count": {},
            "preferences": preferences,
            "resume_filename": resume_filename,
            "resume_text": resume_text,
            "profile": {},
            "jobs_raw": [],
            "jobs_scraped": [],
            "jobs_scored": [],
            "ranking": [],
            "drafts": {},
            "bridge_docs": {},
            "meta": {"created_at_utc": utc_now()},
            "live_feed": [{"layer": "L1", "agent": "API", "message": "Run created. Starting background pipeline…"}],
            "attempts": [],
            "gates": [],
            "evaluations": [],
            "artifacts": {
                "resume_raw": {"path": str(run_dir / "resume_raw.txt"), "content_type": "text/plain"},
                "resume_upload": {"path": str(run_dir / "resume_upload.bin"), "content_type": "application/octet-stream"},
            },
        }

        self._store.upsert_state(run_id=run_id, status=state["status"], state=state, updated_at_utc=utc_now())
        return state

    def get_state(self, run_id: str) -> Optional[Dict[str, Any]]:
        """
        Description: Load run state snapshot.
        Layer: L8
        Input: run_id
        Output: state dict or None
        """
        return self._store.get_state(run_id=run_id)

    def save_state(self, *, run_id: str, state: Dict[str, Any]) -> None:
        """
        Description: Persist run state snapshot.
        Layer: L8
        Input: state dict
        Output: stored snapshot
        """
        self._store.upsert_state(run_id=run_id, status=str(state.get("status","unknown")), state=state, updated_at_utc=utc_now())

    def start_background(self, run_id: str) -> None:
        """
        Description: Start full pipeline in background thread (LangGraph-style node chain).
        Layer: L6
        Input: run_id
        Output: async execution
        """
        t = threading.Thread(target=self._bg_runner, args=(run_id,), daemon=True)
        t.start()

    def _bg_runner(self, run_id: str) -> None:
        # Run async pipeline in dedicated loop
        asyncio.run(self._bg_async(run_id))

    async def _bg_async(self, run_id: str) -> None:
        state = self.get_state(run_id)
        if not state:
            return

        # run the full pipeline (L0-L5). Stops automatically if HITL required.
        state = await run_full_pipeline(state)
        self.save_state(run_id=run_id, state=state)

    async def execute_layer(self, *, run_id: str, layer: str) -> Dict[str, Any]:
        """
        Description: Engineer view — execute a single layer node.
        Layer: L6
        Input: run_id + layer id
        Output: updated state
        """
        state = self.get_state(run_id) or {}
        state = await run_single_layer(state, layer.upper())
        self.save_state(run_id=run_id, state=state)
        return state

    async def handle_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        """
        Description: HITL & operational actions for the UI.
        Layer: L5
        Input: action_type + payload
        Output: updated state
        """
        state = self.get_state(run_id)
        if not state:
            raise ValueError("run_id not found")

        # Store last action for audit
        state.setdefault("meta", {})
        state["meta"]["last_user_action"] = {"type": action_type, "payload": payload, "at_utc": utc_now()}

        # Resume cleanup submit: overwrite resume_text and restart pipeline
        if action_type == "resume_cleanup_submit":
            txt = str(payload.get("resume_text", "")).strip()
            if not txt:
                state["live_feed"].append({"layer":"L5","agent":"HITL","message":"Resume cleanup submitted but empty."})
                self.save_state(run_id=run_id, state=state)
                return state

            # persist new text artifact
            run_dir = self._runs_dir(run_id)
            (run_dir / "resume_raw.txt").write_text(txt, encoding="utf-8")
            state["resume_text"] = txt
            state["artifacts"]["resume_raw"] = {"path": str(run_dir / "resume_raw.txt"), "content_type": "text/plain"}

            state["status"] = "running"
            state["pending_action"] = None
            state["live_feed"].append({"layer":"L2","agent":"HITL","message":"Resume updated. Restarting pipeline…"})

            self.save_state(run_id=run_id, state=state)
            self.start_background(run_id)
            return state

        # Approve / reject individual jobs
        if action_type in ("approve_job", "reject_job"):
            jid = str(payload.get("job_id",""))
            state.setdefault("meta", {})
            state["meta"].setdefault("approved_jobs", [])
            state["meta"].setdefault("rejected_jobs", [])
            if action_type == "approve_job":
                state["meta"]["approved_jobs"].append({"job_id": jid, "at_utc": utc_now()})
                state["live_feed"].append({"layer":"L5","agent":"HITL","message":f"Approved job {jid[:24]}…"})
            else:
                state["meta"]["rejected_jobs"].append({"job_id": jid, "at_utc": utc_now()})
                state["live_feed"].append({"layer":"L5","agent":"HITL","message":f"Rejected job {jid[:24]}…"})
            self.save_state(run_id=run_id, state=state)
            return state

        # Ranking approval -> generate drafts + bridge docs
        if action_type == "approve_ranking":
            state = await approve_ranking_flow(state)
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "reject_ranking":
            reason = str(payload.get("reason","")).strip()
            state.setdefault("meta", {})
            state["meta"].setdefault("refinement_notes", [])
            if reason:
                state["meta"]["refinement_notes"].append(reason)
            state["status"] = "running"
            state["pending_action"] = None
            state["live_feed"].append({"layer":"L5","agent":"HITL","message":"Ranking rejected. Re-running pipeline…"})

            self.save_state(run_id=run_id, state=state)
            self.start_background(run_id)
            return state

        # Draft approval/rejection
        if action_type == "approve_drafts":
            state = await approve_drafts_flow(state)
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "reject_drafts":
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state["live_feed"].append({"layer":"L6","agent":"HITL","message":"Drafts rejected. Returning to ranking review."})
            self.save_state(run_id=run_id, state=state)
            return state

        # Engineer: execute layer
        if action_type == "execute_layer":
            layer = str(payload.get("layer","")).upper()
            state = await run_single_layer(state, layer)
            self.save_state(run_id=run_id, state=state)
            return state

        # Unknown action
        state["live_feed"].append({"layer":"L5","agent":"HITL","message":f"Unhandled action_type={action_type}"})
        self.save_state(run_id=run_id, state=state)
        return state
''')

# ---------- src/careeragent/langgraph/runtime_nodes.py ----------
backup_write("src/careeragent/langgraph/runtime_nodes.py", r'''
from __future__ import annotations

import json
import math
import re
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import httpx
from pydantic_settings import BaseSettings, SettingsConfigDict

from careeragent.agents.parser_agent_service import ParserAgentService
from careeragent.agents.parser_evaluator_service import ParserEvaluatorService

# Optional artifacts root
try:
    from careeragent.config import artifacts_root  # type: ignore
except Exception:
    def artifacts_root() -> Path:
        return Path("src/careeragent/artifacts").resolve()

# ---------------- Settings ----------------
class RuntimeSettings(BaseSettings):
    """
    Description: Runtime settings for tools.
    Layer: L0
    """
    model_config = SettingsConfigDict(env_file=".env", extra="ignore")

    SERPER_API_KEY: Optional[str] = None
    FIRECRAWL_API_KEY: Optional[str] = None
    MCP_SERVER_URL: Optional[str] = None
    MCP_AUTH_TOKEN: Optional[str] = None
    MCP_API_KEY: Optional[str] = None  # supports either naming
    OLLAMA_BASE_URL: Optional[str] = None
    OLLAMA_MODEL: str = "llama3.2"


def mcp_token(s: RuntimeSettings) -> Optional[str]:
    return s.MCP_API_KEY or s.MCP_AUTH_TOKEN


# ---------------- Helpers ----------------
JOB_BOARDS = [
    ("LinkedIn Jobs", "linkedin.com/jobs"),
    ("Indeed", "indeed.com"),
    ("Glassdoor", "glassdoor.com"),
    ("ZipRecruiter", "ziprecruiter.com"),
    ("Monster", "monster.com"),
    ("Dice", "dice.com"),
    ("Lever", "jobs.lever.co"),
    ("Greenhouse", "boards.greenhouse.io"),
]

VISA_NEGATIVE = ("unable to sponsor","cannot sponsor","no sponsorship","do not sponsor","not sponsor","without sponsorship","no visa","cannot provide visa")
RARE_SKILL_SIGNALS = ("langgraph", "mcp", "agentic", "vector db", "faiss", "chroma", "rlhf")

def feed(st: Dict[str, Any], layer: str, agent: str, message: str) -> None:
    st.setdefault("live_feed", [])
    st["live_feed"].append({"layer": layer, "agent": agent, "message": message})

def inc_retry(st: Dict[str, Any], layer: str) -> int:
    st.setdefault("layer_retry_count", {})
    st["layer_retry_count"][layer] = int(st["layer_retry_count"].get(layer, 0)) + 1
    return int(st["layer_retry_count"][layer])

def threshold(st: Dict[str, Any], key: str, default: float = 0.70) -> float:
    t = (st.get("thresholds") or {})
    return float(t.get(key, t.get("default", default)))

def tokenize(text: str) -> List[str]:
    return re.findall(r"[a-zA-Z][a-zA-Z0-9\\+\\#\\.-]{1,}", (text or "").lower())

def cosine(a: Dict[str,int], b: Dict[str,int]) -> float:
    if not a or not b:
        return 0.0
    dot = sum(v * b.get(k,0) for k,v in a.items())
    na = math.sqrt(sum(v*v for v in a.values()))
    nb = math.sqrt(sum(v*v for v in b.values()))
    if na == 0 or nb == 0:
        return 0.0
    return float(dot/(na*nb))

def compute_interview_chance(skill_overlap: float, exp_align: float, ats: float, market: float) -> float:
    market = max(1.0, float(market))
    score = (0.45*skill_overlap + 0.35*exp_align + 0.20*ats) / market
    return max(0.0, min(1.0, float(score)))

def ats_score(text: str) -> float:
    t = (text or "").lower()
    s = 0.0
    if re.search(r"[\\w\\.-]+@[\\w\\.-]+\\.\\w+", t): s += 0.20
    if re.search(r"\\+?\\d[\\d\\-\\s\\(\\)]{8,}\\d", t): s += 0.10
    for h in ["summary","skills","experience","education","projects"]:
        if h in t: s += 0.12
    if "-" in text or "•" in text: s += 0.10
    if len(text) > 1200: s += 0.12
    return max(0.0, min(1.0, s))


async def serper_search(s: RuntimeSettings, query: str, num: int = 10, tbs: Optional[str] = None) -> Tuple[bool, float, Any, Optional[str]]:
    if not s.SERPER_API_KEY:
        return False, 0.0, None, "SERPER_API_KEY missing"
    headers = {"X-API-KEY": s.SERPER_API_KEY, "Content-Type": "application/json"}
    body: Dict[str, Any] = {"q": query, "num": num}
    if tbs:
        body["tbs"] = tbs
    async with httpx.AsyncClient(timeout=30.0) as client:
        r = await client.post("https://google.serper.dev/search", headers=headers, json=body)
    if r.status_code == 403:
        return False, 0.0, None, "Serper 403 quota"
    if r.status_code >= 400:
        return False, 0.0, None, f"Serper {r.status_code}"
    organic = (r.json().get("organic") or [])
    out = [{"title": x.get("title") or "", "link": x.get("link") or "", "snippet": x.get("snippet") or ""} for x in organic]
    conf = 0.75 if out else 0.25
    return True, conf, out, None


async def mcp_invoke(s: RuntimeSettings, tool: str, payload: Dict[str, Any]) -> Tuple[bool, float, Any, Optional[str]]:
    if not (s.MCP_SERVER_URL and mcp_token(s)):
        return False, 0.0, None, "MCP not configured"
    url = s.MCP_SERVER_URL.rstrip("/") + "/invoke"
    headers = {"Authorization": f"Bearer {mcp_token(s)}", "Content-Type": "application/json"}
    async with httpx.AsyncClient(timeout=35.0) as client:
        r = await client.post(url, headers=headers, json={"tool": tool, "payload": payload})
    if r.status_code >= 400:
        return False, 0.0, None, f"MCP {r.status_code}: {r.text[:150]}"
    return True, 0.85, r.json(), None


async def scrape_http(url: str) -> Tuple[bool, float, Any, Optional[str]]:
    try:
        async with httpx.AsyncClient(timeout=18.0, follow_redirects=True) as client:
            r = await client.get(url, headers={"User-Agent":"Mozilla/5.0"})
        if r.status_code >= 400:
            return False, 0.0, None, f"HTTP {r.status_code}"
        html = r.text
        txt = re.sub(r"<(script|style)[^>]*>.*?</\\1>", " ", html, flags=re.S|re.I)
        txt = re.sub(r"<[^>]+>", " ", txt)
        txt = re.sub(r"\\s+", " ", txt).strip()
        conf = 0.65 if len(txt) > 1200 else 0.35
        return True, conf, {"text": txt[:20000]}, None
    except Exception as e:
        return False, 0.0, None, str(e)


def log_attempt(st: Dict[str, Any], *, layer: str, agent: str, tool: str, model: Optional[str], status: str, confidence: float, error: Optional[str]) -> None:
    st.setdefault("attempts", [])
    st["attempts"].append({
        "layer_id": layer,
        "agent": agent,
        "tool": tool,
        "model": model,
        "status": status,
        "confidence": float(confidence),
        "error": error,
    })


def log_gate(st: Dict[str, Any], *, layer: str, target: str, score: float, threshold_v: float, decision: str, feedback: List[str], reasoning_chain: Optional[List[str]] = None) -> None:
    st.setdefault("gates", [])
    st["gates"].append({
        "layer_id": layer,
        "target": target,
        "score": float(score),
        "threshold": float(threshold_v),
        "decision": decision,
        "feedback": feedback,
        "reasoning_chain": reasoning_chain or [],
    })


def gate_decision(score: float, threshold_v: float, retries: int, max_retries: int) -> str:
    if score >= threshold_v:
        return "pass"
    if retries < max_retries:
        return "retry"
    return "hitl"


# ---------------- L0 ----------------
async def L0_security(st: Dict[str, Any]) -> Dict[str, Any]:
    s = RuntimeSettings()
    txt = st.get("resume_text") or ""
    inj = "ignore previous instructions" in txt.lower()
    ok = not inj
    conf = 0.9 if ok else 0.1
    log_attempt(st, layer="L0", agent="SanitizeAgent", tool="local.injection_heuristic", model=None, status=("ok" if ok else "failed"), confidence=conf, error=None if ok else "prompt_injection")
    feed(st, "L0", "SanitizeAgent", "Security passed." if ok else "Blocked: prompt injection detected.")
    if not ok:
        st["status"] = "blocked"
        st["pending_action"] = "security_blocked"
    return st


# ---------------- L2 ----------------
async def L2_parse(st: Dict[str, Any]) -> Dict[str, Any]:
    parser = ParserAgentService()
    txt = st.get("resume_text") or ""
    prof = parser.parse(raw_text=txt, orchestration_state=None, feedback=[])
    st["profile"] = prof.to_json_dict()
    log_attempt(st, layer="L2", agent="ParserAgent", tool="local.regex_parser", model=None, status="ok", confidence=0.65, error=None)
    feed(st, "L2", "ParserAgent", "Intake bundle created.")
    return st


async def EVAL_parser(st: Dict[str, Any]) -> Dict[str, Any]:
    prefs = st.get("preferences") or {}
    th = float((st.get("thresholds") or {}).get("parser", prefs.get("resume_threshold", 0.70)))
    prof = st.get("profile") or {}
    skills = len(prof.get("skills") or [])
    contact = prof.get("contact") or {}
    has_contact = bool(contact.get("email") or contact.get("phone"))
    score = min(1.0, 0.35 + (skills / 30.0) + (0.15 if has_contact else 0.0))
    fb = []
    if not has_contact:
        fb.append("Missing contact info. Continue, but ATS + recruiter trust improves with email/phone.")
    if skills < 8:
        fb.append("Skill density low. Add more relevant tools/keywords.")

    retries = int((st.get("layer_retry_count") or {}).get("L2", 0))
    decision = gate_decision(score, th, retries, int(st.get("max_retries", 3)))
    log_gate(st, layer="L2", target="parser", score=score, threshold_v=th, decision=decision, feedback=fb)

    feed(st, "L3", "EvaluatorAgent", f"Parser score={score:.2f} decision={decision}")
    if decision == "hitl":
        st["status"] = "needs_human_approval"
        st["pending_action"] = "resume_cleanup_optional"
        st["hitl_reason"] = "Parser threshold not met"
        st["hitl_payload"] = {"feedback": fb, "score": score, "threshold": th}
    elif decision == "retry":
        inc_retry(st, "L2")
    return st


# ---------------- L3 Discovery (3 tools) ----------------
async def L3_discovery(st: Dict[str, Any]) -> Dict[str, Any]:
    s = RuntimeSettings()
    prefs = st.get("preferences") or {}
    roles = prefs.get("target_roles") or [prefs.get("target_role") or "Data Scientist"]
    roles = [r.strip() for r in roles if str(r).strip()][:4]
    location = str(prefs.get("location","United States"))
    visa_req = bool(prefs.get("visa_sponsorship_required", False))
    recency_h = float(prefs.get("recency_hours", 36))
    tbs = "qdr:d" if recency_h <= 36 else None

    prof = st.get("profile") or {}
    skills_hint = " ".join((prof.get("skills") or [])[:6])

    def build_query(role: str) -> str:
        visa_part = '"visa sponsorship" OR h1b OR opt OR cpt' if visa_req else ""
        return f'{role} {location} {skills_hint} ({visa_part}) apply'

    queries = [build_query(r) for r in roles]
    st["discovery_queries"] = queries

    # Tool A: Serper
    all_hits: List[Dict[str, Any]] = []
    serper_ok = True
    for q in queries:
        ok, conf, data, err = await serper_search(s, q, num=20, tbs=tbs)
        log_attempt(st, layer="L3", agent="DiscoveryAgent", tool="serper.search", model=None,
                    status=("ok" if ok and conf >= 0.55 else ("low_conf" if ok else "failed")),
                    confidence=conf, error=err)
        if ok:
            for it in data:
                it["query"] = q
                it["source"] = "serper"
            all_hits.extend(data)
        else:
            serper_ok = False

    # Tool B: MCP fallback
    if (not all_hits) or (not serper_ok):
        ok2, conf2, data2, err2 = await mcp_invoke(s, "jobs.search", {"queries": queries, "recency_hours": recency_h})
        log_attempt(st, layer="L3", agent="DiscoveryAgent", tool="mcp.jobs.search", model=None,
                    status=("ok" if ok2 and conf2 >= 0.55 else ("low_conf" if ok2 else "failed")),
                    confidence=conf2, error=err2)
        if ok2 and isinstance(data2, dict):
            hits = data2.get("results") or data2.get("jobs") or []
            for it in hits:
                it["source"] = "mcp"
            all_hits.extend(hits)

    # Dedup by link
    seen = set()
    uniq = []
    for it in all_hits:
        link = it.get("link") or it.get("url") or ""
        if link and link not in seen:
            seen.add(link)
            # normalize keys
            uniq.append({"title": it.get("title") or "", "link": link, "snippet": it.get("snippet") or "", "source": it.get("source") or "unknown"})
    st["jobs_raw"] = uniq
    feed(st, "L3", "DiscoveryAgent", f"Found {len(uniq)} unique jobs across boards.")
    return st


async def EVAL_discovery(st: Dict[str, Any]) -> Dict[str, Any]:
    prefs = st.get("preferences") or {}
    th = float((st.get("thresholds") or {}).get("discovery", prefs.get("discovery_threshold", 0.70)))
    n = len(st.get("jobs_raw") or [])
    score = 0.2 if n < 8 else (0.6 if n < 20 else 0.85)
    fb = []
    if n < 20:
        fb.append("Low job volume after filters. Broaden roles, raise recency window, or loosen visa filter.")
    retries = int((st.get("layer_retry_count") or {}).get("L3", 0))
    decision = gate_decision(score, th, retries, int(st.get("max_retries", 3)))
    log_gate(st, layer="L3", target="discovery", score=score, threshold_v=th, decision=decision, feedback=fb)
    feed(st, "L5", "EvaluatorAgent", f"Discovery score={score:.2f} decision={decision}")
    if decision == "hitl":
        st["status"] = "needs_human_approval"
        st["pending_action"] = "discovery_low_confidence"
        st["hitl_reason"] = "Discovery threshold not met"
        st["hitl_payload"] = {"feedback": fb, "score": score, "threshold": th}
    elif decision == "retry":
        inc_retry(st, "L3")
    return st


# ---------------- L4 Scrape+Match (3 tools scrape + scoring) ----------------
async def L4_match(st: Dict[str, Any]) -> Dict[str, Any]:
    s = RuntimeSettings()
    prefs = st.get("preferences") or {}
    max_jobs = int(prefs.get("max_jobs", 40))
    visa_req = bool(prefs.get("visa_sponsorship_required", False))

    prof = st.get("profile") or {}
    resume_text = st.get("resume_text") or ""
    resume_skills = [str(x).lower() for x in (prof.get("skills") or [])]
    ats = ats_score(resume_text)

    exp_text = ""
    exp = prof.get("experience") or []
    if isinstance(exp, list) and exp:
        # try bullets
        e0 = exp[0] if isinstance(exp[0], dict) else {}
        bullets = e0.get("bullets") if isinstance(e0, dict) else []
        if isinstance(bullets, list):
            exp_text = " ".join(bullets)
    exp_text = exp_text or resume_text
    exp_tokens = {}
    for t in tokenize(exp_text):
        exp_tokens[t] = exp_tokens.get(t, 0) + 1

    jobs = (st.get("jobs_raw") or [])[:max_jobs]
    scored: List[Dict[str, Any]] = []

    for idx, j in enumerate(jobs):
        url = j.get("link") or ""
        snippet = j.get("snippet") or ""
        title = j.get("title") or ""
        source = j.get("source") or "unknown"

        # Tool A: HTTP scrape
        ok, conf, data, err = await scrape_http(url)
        log_attempt(st, layer="L4", agent="ScraperAgent", tool="httpx.scrape", model=None,
                    status=("ok" if ok and conf >= 0.45 else ("low_conf" if ok else "failed")),
                    confidence=conf, error=err)
        text = (data or {}).get("text") if ok else ""

        # Tool B: MCP scrape if low confidence
        if (not ok) or conf < 0.45 or not text:
            ok2, conf2, data2, err2 = await mcp_invoke(s, "web.scrape", {"url": url})
            log_attempt(st, layer="L4", agent="ScraperAgent", tool="mcp.web.scrape", model=None,
                        status=("ok" if ok2 and conf2 >= 0.45 else ("low_conf" if ok2 else "failed")),
                        confidence=conf2, error=err2)
            if ok2 and isinstance(data2, dict):
                text = data2.get("text") or data2.get("content") or text

        text = text or snippet
        low = text.lower()
        visa_ok = not any(x in low for x in VISA_NEGATIVE)
        if visa_req and not visa_ok:
            continue

        # Score locally (stable & explainable)
        matched = [s for s in resume_skills if s and s in low][:30]
        skill_overlap = (len(set(matched)) / max(1, len(set(resume_skills)))) if resume_skills else 0.0

        job_tokens = {}
        for t in tokenize(text):
            job_tokens[t] = job_tokens.get(t, 0) + 1
        exp_align = cosine(exp_tokens, job_tokens)

        market = 1.0
        score = compute_interview_chance(skill_overlap, exp_align, ats, market)

        # Missing skills heuristic: detect common skill words in JD not in resume_skills
        missing = []
        for kw in ["langgraph","mcp","kubernetes","mlflow","dvc","airflow","kafka","terraform","databricks","snowflake","faiss","chroma"]:
            if kw in low and kw not in resume_skills:
                missing.append(kw)

        scored.append({
            "job_id": url or f"job_{idx}",
            "title": title,
            "url": url,
            "source": source,
            "snippet": snippet,
            "full_text": text[:8000],
            "visa_ok": visa_ok,
            "components": {
                "skill_overlap": float(skill_overlap),
                "experience_alignment": float(exp_align),
                "ats_score": float(ats),
                "market_competition_factor": float(market),
            },
            "missing_skills": missing[:12],
            "score": float(score),
            "match_percent": round(float(score) * 100.0, 2),
        })

    scored.sort(key=lambda x: float(x["score"]), reverse=True)
    st["jobs_scored"] = scored
    feed(st, "L4", "MatcherAgent", f"Scored {len(scored)} jobs.")
    return st


async def EVAL_match(st: Dict[str, Any]) -> Dict[str, Any]:
    prefs = st.get("preferences") or {}
    th = float((st.get("thresholds") or {}).get("match", prefs.get("discovery_threshold", 0.70)))
    jobs = st.get("jobs_scored") or []
    top = float(jobs[0]["score"]) if jobs else 0.0
    score = top
    fb = []
    if top < th:
        fb.append("Top match low. Try alternate role titles or improve resume keyword alignment.")
    retries = int((st.get("layer_retry_count") or {}).get("L4", 0))
    decision = gate_decision(score, th, retries, int(st.get("max_retries", 3)))
    log_gate(st, layer="L4", target="match", score=score, threshold_v=th, decision=decision, feedback=fb)
    feed(st, "L5", "EvaluatorAgent", f"Match top={top:.2f} decision={decision}")
    if decision == "hitl":
        st["status"] = "needs_human_approval"
        st["pending_action"] = "review_ranking"
        st["hitl_reason"] = "Ranking ready or match threshold not met"
        st["hitl_payload"] = {"feedback": fb, "score": score, "threshold": th}
    elif decision == "retry":
        inc_retry(st, "L4")
    return st


# ---------------- L5 Rank + High Interview Potential bypass ----------------
async def L5_rank(st: Dict[str, Any]) -> Dict[str, Any]:
    prefs = st.get("preferences") or {}
    top_n = int(prefs.get("top_n", 30))
    min_accept = float(prefs.get("min_match", 0.50))
    jobs = st.get("jobs_scored") or []
    if not jobs:
        st["status"] = "needs_human_approval"
        st["pending_action"] = "no_jobs"
        return st

    ranked = []
    flagged = []
    reasoning_chain = []

    for j in jobs[:max(50, top_n)]:
        s = float(j["score"])
        if s >= min_accept:
            ranked.append(j)
        else:
            text = (j.get("full_text") or "").lower()
            if any(x in text for x in RARE_SKILL_SIGNALS):
                reasoning_chain = [
                    f"Match score {s:.2f} < 0.50, but rare-skill signal detected in job.",
                    "Rare skills reduce applicant pool and increase interview odds.",
                    "Bypassing rejection → flagged for HITL review."
                ]
                flagged.append(j)

    ranked.sort(key=lambda x: float(x["score"]), reverse=True)
    st["ranking"] = ranked[:top_n]
    st.setdefault("hitl_payload", {})
    st["hitl_payload"]["flagged_low_score_high_potential"] = flagged[:20]
    st["hitl_payload"]["reasoning_chain"] = reasoning_chain

    st["status"] = "needs_human_approval"
    st["pending_action"] = "review_ranking"
    feed(st, "L5", "Ranker", f"Ranking ready: {len(st['ranking'])} jobs. Flagged={len(flagged)}")
    return st


# ---------------- Public runners ----------------
async def run_full_pipeline(st: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: Run L0->L5 chain, stopping on HITL/blocked.
    Layer: L6
    """
    st["status"] = "running"
    st["pending_action"] = None
    feed(st, "L1", "Orchestrator", "Automation active: running L0→L5…")

    st = await L0_security(st)
    if st.get("status") in ("blocked", "needs_human_approval"):
        return st

    st = await L2_parse(st)
    st = await EVAL_parser(st)
    # parser HITL is OPTIONAL; continue anyway if not blocked
    if st.get("status") == "blocked":
        return st

    st = await L3_discovery(st)
    st = await EVAL_discovery(st)
    if st.get("status") in ("blocked", "needs_human_approval"):
        return st

    st = await L4_match(st)
    st = await EVAL_match(st)
    # If evaluator sets HITL, we stop here and let user approve ranking
    if st.get("status") in ("blocked", "needs_human_approval"):
        # If match evaluator didn't set review_ranking, run rank anyway
        if st.get("pending_action") not in ("review_ranking",):
            st = await L5_rank(st)
        return st

    # Proceed to ranking
    st = await L5_rank(st)
    return st


async def run_single_layer(st: Dict[str, Any], layer: str) -> Dict[str, Any]:
    """
    Description: Execute a single layer (Engineer View).
    Layer: L6
    """
    layer = layer.upper()
    if layer == "L0":
        return await L0_security(st)
    if layer == "L2":
        st = await L2_parse(st)
        return await EVAL_parser(st)
    if layer == "L3":
        st = await L3_discovery(st)
        return await EVAL_discovery(st)
    if layer == "L4":
        st = await L4_match(st)
        return await EVAL_match(st)
    if layer == "L5":
        return await L5_rank(st)

    feed(st, "L1", "Engineer", f"Layer {layer} not implemented in runtime_nodes yet.")
    return st


# ---------------- HITL flows ----------------
def _runs_dir(run_id: str) -> Path:
    d = artifacts_root() / "runs" / run_id
    d.mkdir(parents=True, exist_ok=True)
    return d


async def approve_ranking_flow(st: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: L6 draft generation + L9 bridge docs.
    Layer: L6/L9
    """
    run_id = st.get("run_id","")
    run_dir = _runs_dir(run_id)
    prof = st.get("profile") or {}
    ranking = st.get("ranking") or []
    top_n = int((st.get("preferences") or {}).get("draft_count", 10))
    ranking = ranking[:top_n]

    # Base ATS resume template built from intake
    name = str(prof.get("name") or "Candidate")
    contact = prof.get("contact") or {}
    email = str(contact.get("email") or "")
    phone = str(contact.get("phone") or "")
    linkedin = str(contact.get("linkedin") or "")
    github = str(contact.get("github") or "")
    skills = ", ".join((prof.get("skills") or [])[:25])

    base_resume = f"""# {name}
{email} | {phone} | {linkedin} | {github}

## Summary
AI/ML + GenAI builder focused on production-grade delivery (MLOps, evaluation, governance).

## Skills
{skills}

## Experience
- Add 4–6 impact bullets per role with metrics, scale, tools, and outcomes.

## Education
- (from intake)
"""
    (run_dir / "ats_resume_base.md").write_text(base_resume, encoding="utf-8")
    st.setdefault("artifacts", {})
    st["artifacts"]["ats_resume_base"] = {"path": str(run_dir / "ats_resume_base.md"), "content_type": "text/markdown"}

    # Draft packages + learning plan
    drafts: List[Dict[str, Any]] = []
    learning_plan: Dict[str, Any] = {}

    settings = RuntimeSettings()

    # Reuse Serper for learning links
    async def serper_links(skill: str, q: str) -> List[Dict[str, Any]]:
        ok, conf, data, err = await serper_search(settings, q, num=3, tbs=None)
        if not ok or not data:
            return []
        return [{"title": x.get("title"), "link": x.get("link"), "snippet": x.get("snippet")} for x in data[:3]]

    for j in ranking:
        jid = str(j.get("job_id") or j.get("url") or j.get("title"))
        title = str(j.get("title") or "Role")
        company = str(j.get("source") or j.get("board") or "Company")
        url = str(j.get("url") or "")

        matched = (j.get("matched_skills") or [])[:12]
        missing = (j.get("missing_skills") or [])[:10]

        tailored = base_resume + f"\n\n## Target Role Alignment\n- Target: {title} @ {company}\n- Matched Keywords: {', '.join(matched)}\n- Gap Keywords: {', '.join(missing)}\n"
        resume_path = run_dir / f"resume_{jid[:10]}.md"
        resume_path.write_text(tailored, encoding="utf-8")

        cover = f"""{name}
{email}

Dear Hiring Manager,

I’m applying for the {title} role. I bring production-grade AI/ML and GenAI experience, including model evaluation, MLOps pipelines, and reliable API deployments.

Keywords aligned to this role: {', '.join(matched[:8])}.

I’d welcome a conversation about how I can help {company} deliver measurable AI impact.

Sincerely,
{name}
"""
        cover_path = run_dir / f"cover_{jid[:10]}.md"
        cover_path.write_text(cover, encoding="utf-8")

        drafts.append({
            "job_id": jid,
            "title": title,
            "company": company,
            "url": url,
            "resume_path": str(resume_path),
            "cover_path": str(cover_path),
            "missing_skills": missing,
        })

        if missing and settings.SERPER_API_KEY:
            plan = {}
            for sk in missing[:8]:
                plan[sk] = {
                    "youtube": await serper_links(sk, f"{sk} tutorial youtube"),
                    "docs": await serper_links(sk, f"{sk} official documentation"),
                    "course": await serper_links(sk, f"best course learn {sk}"),
                }
            learning_plan[jid] = plan

    bundle = {"drafts": drafts, "learning_plan": learning_plan}
    (run_dir / "drafts_bundle.json").write_text(json.dumps(bundle, indent=2), encoding="utf-8")
    st["drafts"] = bundle
    st["artifacts"]["drafts_bundle"] = {"path": str(run_dir / "drafts_bundle.json"), "content_type": "application/json"}

    feed(st, "L6", "DraftAgent", f"Generated {len(drafts)} draft packages + learning plans.")
    st["status"] = "needs_human_approval"
    st["pending_action"] = "review_drafts"
    return st


async def approve_drafts_flow(st: Dict[str, Any]) -> Dict[str, Any]:
    """
    Description: L7 apply simulation + mark completed.
    Layer: L7
    """
    feed(st, "L7", "ApplyExecutor", "Drafts approved. Simulating submissions…")
    st.setdefault("meta", {})
    st["meta"].setdefault("applied_jobs", [])
    drafts = (st.get("drafts") or {}).get("drafts") or []
    for d in drafts[:10]:
        st["meta"]["applied_jobs"].append({
            "job_id": d.get("job_id"),
            "title": d.get("title"),
            "company": d.get("company"),
            "url": d.get("url"),
            "status": "Applied",
        })
    st["status"] = "completed"
    st["pending_action"] = None
    feed(st, "L9", "Analytics", "Run completed. Applied jobs recorded.")
    return st
''')

# ---------- src/careeragent/api/main.py ----------
backup_write("src/careeragent/api/main.py", r'''
from __future__ import annotations

import json
from typing import Any, Dict, Optional

from fastapi import FastAPI, File, Form, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware

from careeragent.api.request_models import AnalyzeResponse, ActionRequest
from careeragent.api.run_manager_service import RunManagerService


app = FastAPI(title="CareerAgent-AI Brain", version="0.2.0")

# CORS for Streamlit local UI
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

RM = RunManagerService()


@app.get("/health")
def health() -> Dict[str, Any]:
    """
    Description: Health endpoint.
    Layer: L0
    Input: None
    Output: status JSON
    """
    return {"status": "ok"}


def _extract_text(filename: str, data: bytes) -> str:
    """
    Description: Extract text from resume file upload.
    Layer: L2
    Input: filename + bytes
    Output: raw text
    """
    name = (filename or "").lower()
    if name.endswith(".txt"):
        return data.decode("utf-8", errors="replace")
    if name.endswith(".pdf"):
        try:
            from pypdf import PdfReader  # type: ignore
            import io
            reader = PdfReader(io.BytesIO(data))
            return "\n".join([(pg.extract_text() or "") for pg in reader.pages])
        except Exception:
            return data.decode("utf-8", errors="replace")
    if name.endswith(".docx"):
        try:
            import docx  # type: ignore
            import io
            d = docx.Document(io.BytesIO(data))
            return "\n".join([p.text for p in d.paragraphs if p.text])
        except Exception:
            return data.decode("utf-8", errors="replace")
    return data.decode("utf-8", errors="replace")


@app.post("/analyze", response_model=AnalyzeResponse)
async def analyze(
    resume: UploadFile = File(...),
    preferences_json: str = Form(...),
) -> AnalyzeResponse:
    """
    Description: Start a new run (background pipeline).
    Layer: L1-L6
    Input: resume upload + preferences_json
    Output: run_id + status
    """
    try:
        prefs = json.loads(preferences_json)
    except Exception:
        raise HTTPException(status_code=400, detail="Invalid preferences_json")

    data = await resume.read()
    text = _extract_text(resume.filename or "resume.txt", data)

    state = RM.create_run(
        resume_filename=resume.filename or "resume.txt",
        resume_text=text,
        resume_bytes=data,
        preferences=prefs,
    )

    RM.start_background(state["run_id"])
    return AnalyzeResponse(run_id=state["run_id"], status=state["status"])


@app.get("/status/{run_id}")
def status(run_id: str) -> Dict[str, Any]:
    """
    Description: Poll run state.
    Layer: L8
    Input: run_id
    Output: full state JSON
    """
    st = RM.get_state(run_id)
    if not st:
        raise HTTPException(status_code=404, detail="run_id not found")
    return st


@app.post("/action/{run_id}")
async def action(run_id: str, req: ActionRequest) -> Dict[str, Any]:
    """
    Description: Handle engineer actions + HITL approvals.
    Layer: L5
    Input: ActionRequest
    Output: updated state
    """
    try:
        st = await RM.handle_action(run_id=run_id, action_type=req.action_type, payload=req.payload)
        return st
    except ValueError:
        raise HTTPException(status_code=404, detail="run_id not found")
''')

print("\n✅ FastAPI /action + background LangGraph-style runtime written.")
print("Next: restart backend + UI.\n")
print("Backend:")
print("  PYTHONPATH=src uv run python -m uvicorn careeragent.api.main:app --host 127.0.0.1 --port 8000 --reload")
print("UI:")
print("  API_URL=http://127.0.0.1:8000 PYTHONPATH='.:src' uv run streamlit run app/main.py --server.port 8501")